[【Python】企業の決算情報をEDINET APIで取得する](https://non-dimension.com/get-xbrldata/)

In [ ]:
# import
import requests
import json

In [ ]:
# 書類一覧APIのエンドポイント
url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
# 書類一覧APIのリクエストパラメータ
params = {
  "date" : '2019-06-26',
  "type" : 2
}

# 書類一覧APIの呼び出し
res = requests.get(url, params=params, verify=False)

# レスポンス（JSON）の表示
# print(res.text)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
#APIの結果をjson形式に変換
res_text = json.loads(res.text)

#res_text内のresultsの内容を取得
results= res_text["results"]

#resultsの中身（docID, docDescription, filerName）を表示
# for result in results:
#     print(result['docID'], result['docDescription'],result['filerName'])

In [ ]:
kessan = []
for result in results:
    if result['docDescription'] is not None:
        if '松風' in result['filerName']:
            if '有価証券報告書' in result['docDescription']:
                print(result['docID'], result['docDescription'],result['filerName'])
                kessan.append(result)

S100FXAC 有価証券報告書－第147期(平成30年4月1日－平成31年3月31日) 株式会社松風


In [ ]:
#0番目のdocumentのdocID（S100FOJX）を取得
docid = kessan[0]['docID'] 
print(docid)

S100FXAC


In [ ]:
# 書類取得APIのエンドポイント
url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + docid

# 書類取得APIのリクエストパラメータ
params = {
  "type" : 1
}

# 出力ファイル名
filename = docid + ".zip"

# 書類取得APIの呼び出し
res = requests.get(url, params=params, verify=False)

# ファイルへ出力
if res.status_code == 200:
  with open(filename, 'wb') as f:
    for chunk in res.iter_content(chunk_size=1024):
      f.write(chunk)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


### zipファイルの解凍

In [ ]:
import zipfile
with zipfile.ZipFile(filename) as existing_zip:
    existing_zip.extractall(docid)

### htmlファイルの取得

In [ ]:
import glob

filepath = docid + '/XBRL/PublicDoc/'

files = glob.glob(filepath+'*.htm') #htmファイルの取得
files = sorted(files) #ファイルを並び替えているだけ
target_file = files[6]
print(target_file)

S100FXAC/XBRL/PublicDoc/0105010_honbun_jpcrp030000-asr-001_E01183-000_2019-03-31_01_2019-06-26_ixbrl.htm


In [ ]:
with open(target_file , encoding='utf-8') as f:
    html = f.read()

In [ ]:
html

'\ufeff<?xml version="1.0" encoding="utf-8"?>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:jpcrp_cor="http://disclosure.edinet-fsa.go.jp/taxonomy/jpcrp/2018-03-31/jpcrp_cor" xmlns:jpcrp030000-asr_E01183-000="http://disclosure.edinet-fsa.go.jp/jpcrp030000/asr/001/E01183-000/2019-03-31/01/2019-06-26" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:jppfs_cor="http://disclosure.edinet-fsa.go.jp/taxonomy/jppfs/2018-03-31/jppfs_cor" xmlns:jpdei_cor="http://disclosure.edinet-fsa.go.jp/taxonomy/jpdei/2013-08-31/jpdei_cor" xmlns:ix="http://www.xbrl.org/2008/inlineXBRL" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2011-07-31">\n<head>\n<title>0105010_honbun_0275500103104.htm</title>\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n<

### BeautifulSoupでhtml内のデータを取得

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

tag_p = soup.find_all('p') #pタグの要素を取得

#pタグのテキストを表示
# for p in tag_p:
#     print(p.text)

In [ ]:
p_list = []
for p in tag_p:
    p_list.append(p.text)
p_list